# **Regression Modelling Notebook**

## Objectives

* Train ML Pipeline using hyperparameter optimization with the best features
* Fit and evaluate a regression model to predict house sale price in Ames, Iowa.

## Inputs

* outputs/datasets/collection/HousePriceRecords.csv 

## Outputs

* Train set (features and target)
* Test set (features and target)
* ML pipeline to predict house sale price
* Feature Importance Plot

## Additional Comments

 **CRISP-DM Workflow**

* Modelling and Evaluation


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

Load data from the collection folder so that the pipeline will be able to handle the cleaning and engineering by itself

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")
df.head(3)

## ML Pipeline: Regression

We create ML piplines required for data cleaning and feature engineering

In [ ]:
from sklearn.pipeline import Pipeline

# Data Cleaning
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer

# Feature Engineering
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer

# Feature Scaling
from sklearn.preprocessing import StandardScaler

# Feature Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

def PipelineRegression(model):
    pipeline_base = Pipeline([

        # Data Cleaning
        ("MedianImputation", MeanMedianImputer(imputation_method='median', variables=['2ndFlrSF', 'BedroomAbvGr', 'EnclosedPorch', 'GarageYrBlt','MasVnrArea', 'WoodDeckSF'])),

        ("MeanImputation", MeanMedianImputer(imputation_method='mean', variables=['LotFrontage'])),

        ("CategoricalImputer", CategoricalImputer(imputation_method='frequent', variables=['BsmtFinType1', 'GarageFinish'])),

        # Feature Engineering
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary', variables=['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])),

        ("LogTransformer", vt.LogTransformer(variables=['1stFlrSF', 'GrLivArea', 'LotArea'])),

        ("PowerTransformer", vt.PowerTransformer(variables=['BsmtFinSF1', 'BsmtUnfSF', 'GarageArea'])),
        
        ("YeoJohnsonTransformer", vt.YeoJohnsonTransformer(
                         variables = ['GarageYrBlt', 'LotFrontage', 'OpenPorchSF', 'TotalBsmtSF', 'YearBuilt']) ),
        
        ("Winsorizer", Winsorizer(capping_method='iqr', tail='both', fold=1.5,
                        variables = ['1stFlrSF', 'GrLivArea', 'LotArea', 'LotFrontage', 'TotalBsmtSF']) ),

        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.7, selection_method="variance")),

        # Feature Scaling
        ("feat_scaler", StandardScaler()),

        # Feature Selection
        ("feat_selection", SelectFromModel(model)),

        # ML Algorithms
        ("model", model),
        
      ])

    return pipeline_base

Custom Python Class for hyperparameter optimisation, where we parse the models and hyperparameters for each model
Note: 

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineRegression(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs
            
    # Returns all pipelines, and a DataFrame with a performance summary for the algorithms.
    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

## Split Train Test Set

We split the data set into the train and test sets, with a standard 20% in the test size and a random state of zero and using all features.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

## Grid Search CV - Sklearn

We will do an extensive hyperparameter search using the default hyperparameters to find most suitable algorithm.
Then we define the hyperparameters for each algorithm using an empty dictionary

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

We parsed the train set and set the performance metric as an R2 score and set cross validation as 5 which we have described in the ML business case.

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check the performance summary, and we can see, the best result are LinearRegression and ExtraTreesRegressor.

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

### Do an extensive search on the algorithms that performed better in a different hyperparameter optimization

We select the LinearRegression and the ExtraTreesRegressor algorithm for the extensive hyperparameter search with different ranges for the hyperparameter values

In [ ]:
models_search = {
    'LinearRegression': LinearRegression(),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
}



# documentation to help on hyperparameter list:

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
# No specific hyperparameters:  You should parse an empty dictionary

# Rationale: Linear regression is a relatively simple model that does not have many hyperparameters to tune.
# The primary focus is often on feature engineering, data preprocessing,
# and handling potential issues like multicollinearity or outliers.



# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html

# n_estimators int, default=100
# * Number of estimators (or trees)
# * Typical values: 100 to 1000
# * Rationale: Increasing the number of estimators allows the model to capture more complex relationships in the data.
#* A larger number of estimators may increase training time, so the value should be chosen based on computational resources.


# max_depth: int or None, default=None
# * Maximum depth of trees
# * Typical values: 5 to 10
# * Rationale: Setting max_depth to a small value helps control overfitting and keeps the individual trees shallow.
# * This can improve generalization and prevent the model from capturing noise or outliers in the training data.


# min_samples_split: int or float, default=2
# * Typical values: 2 to 10 or higher
# * Rationale: A small value like 2 allows splitting at every internal node, 
  # * but setting a higher value can prevent overfitting by requiring a minimum number of samples in each split.


# min_samples_leaf: int or float, default=1
# * Typical values: 1 to 10 or higher
# * Rationale: A small value like 1 allows individual samples to be leaf nodes.
# * Increasing this value can prevent overfitting and promote simpler, more generalized trees.



# max_samples: int or float, default=None
# * Typical values: value less than 1.0 (e.g., 0.8)
# * Rationale: The parameter controls the maximum number of samples to be used for training each base estimator in the ensemble. 
  # *By default, it uses all samples. However, setting a value less than 1.0 randomly selects a subset of samples for each tree,
  # which can introduce more diversity and reduce overfitting, especially in the presence of large datasets.



# **max_features {“sqrt”, “log2”, None}, int or float, default=1.0**
# * Typical values: "auto" (default), "sqrt", "log2", or a specific value like 0.5
# * Rationale: selecting a smaller subset of features is to introduce more randomness
# and reduce the correlation between individual trees in the ensemble,
# which can help prevent overfitting and improve generalization.



params_search = {
    'LinearRegression': {},
    "ExtraTreesRegressor":{'model__n_estimators': [100, 150, 400],
                                  'model__max_depth': [3,10, None],
                                  'model__min_samples_split': [2, 10, 20],
                                  'model__min_samples_leaf': [1, 10],
                                  'model__max_samples': [None, 0.8],
                                  'model__max_features':[0.5],
                            }
  }

Perforn an extensive search using GridSearch Cross Validation across the hyperparameters:

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

 check the results summary with .score_summary

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

The extensive (GridSearchCV) search only display estimator for ExtraTreesRegressor and has made slight improvement of the mean_score from 0.804232 to 0.834859

We check the best model name, by using .iloc[]

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

We get the best model parameters

In [ ]:
grid_search_pipelines[best_model].best_params_

We subset the pipelines from the algorithm having the best performance (with best_model), then use .best_estimator_ to retrieve the pipeline that has the algorithm and hyperparameter configuration

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

### Assess feature importance

We can check the most important features with .feature_importances_
Note: This has been adapted from the Code Institute learning material Scikit-Learn Unit 6: Cross Validation Search Part 2.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# after data cleaning and feature engineering, the features may have changes
# how many data cleaning and feature engineering steps does your pipeline have?
data_cleaning_feat_eng_steps = 9
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support(
)].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
    'Importance': best_regressor_pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

The Pipeline selected 5 features out of 23 to train the model: 'OverallQual', 'GrLiveArea', 'TotalBsmtSF', 'GarageArea', and 'GarageYrBlt'.

## Evaluate on Train and Test Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np


def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(
        mean_squared_error(y, prediction)).round(3))
    print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    plt.show()

Evaluate Performance

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train,
                            X_test, y_test,
                            best_regressor_pipeline)

* We notice that the performance on the train set is pretty good 0.912 of R2
* We notice the performance on the train and test set are not too different. 
* Which indicates that the model didn't overfit.
* The test set performance which was used to simulate real data since the model has never seen, has an R2 performance of 0.806
* We also note in the plots that Prediction x Actual plot, the predictions tend to follow the actual value(the red diagonal line)
* The pipeline was trained on the algorithm hyperparameter combination with values (estimators, max-depth, min-samples-split, min-samples-leaf, max_samples and max_features)
* In conclusion this is over the 0.75 client's requirement, which indicates the model can provide good predictions.

---

## Refit pipeline with best features

We rewrite the ML Pipeline using only the selected 5 most important features
* Now, we only have 1 step for data cleaning and 4 for feature engineering.

In [ ]:
# Pipeline Optmization: Model

def PipelineRegression(model):
    pipeline_base = Pipeline([

        # Data Cleaning
        ("MedianImputation", MeanMedianImputer(imputation_method='median', variables=['GarageYrBlt'])),

        # Feature Engineering
        ("LogTransformer", vt.LogTransformer(variables=['GrLivArea'])),
        
        ("PowerTransformer", vt.PowerTransformer(variables=['GarageArea'])),
        
        ("YeoJohnsonTransformer", vt.YeoJohnsonTransformer(
                         variables = ['TotalBsmtSF']) ),
        
        ("Winsorizer", Winsorizer(capping_method='iqr', tail='both', fold=1.5,
                        variables = ['TotalBsmtSF']) ),
        
        # feature smart correlation is not needed. We know which features to use already!
        
        # Feature Scaling
        ("feat_scaler", StandardScaler()),

        # feature selection is not needed. We know which features to use already!
        
        # ML Algorithms
        ("model", model),
        
      ])

    return pipeline_base

## Split Train Test Sets, only with the best features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                    df.drop(['SalePrice'], axis=1),
                                    df['SalePrice'],
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",X_test.shape, y_test.shape)

We Subset the datasets to only include the best features

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:", X_test.shape, y_test.shape)
X_train.head()

We list the model that performed best with the best parameters

In [ ]:
models_search = {
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
}

params_search = {
    "ExtraTreesRegressor":{'model__n_estimators': [400],
                                  'model__max_depth': [None],
                                  'model__min_samples_split': [10],
                                  'model__min_samples_leaf': [1],
                                  'model__max_features': [0.5],
                                  'model__max_samples': [None]}
}


We perform HyperparameterOptimizationSearch considering the model "ExtraTreesRegressor" and best hyperparameter configuration we discovered.

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search,
                                          params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

we check the search summary

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

 There's a slight drop from the previous pipeline of 0.834859 mean_score

We get the best model

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

We subset the pipelines from the algorithm having the best performance (with best_model), to grab the pipline

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

### Assess feature importance

In [ ]:
data_cleaning_feat_eng_steps = 5

columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng


# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
                        'Feature': best_features,
                        'Importance': best_regressor_pipeline['model'].feature_importances_})
                    .sort_values(by='Importance', ascending=False)
        )

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

### Evaluate Regression Pipeline on Train and Test Sets

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train,
                            X_test, y_test,
                            best_regressor_pipeline)

* The performance from this pipeline is more of the same as from the previous pipeline (The R2 scores for the Train Set is 0.912, while the for the Test sets is 0.806).
* The R2 scores for the Train Set is 0.913, while the for the Test sets is 0.81.
* The 5 features performed the same as the test performed on all features to predict sale price. Also, it's above the 0.75 client's requirement. Therefore, this pipline will be deployed.
* The R2 score between train and test sets, indicate the model does not overfit.

# Push files to Repo

We will generate the following files

* Train set
* Test set
* Modeling pipeline
* features importance plot

The version was changed due to SmartCorrelationSelection threshold error in the ML Pipeline. Threshold value was different from the value in the Smart Correlated Selection in feature engineering notebook.

In [ ]:
import joblib
import os

version = 'v2'
file_path = f'outputs/ml_pipeline/predict_saleprice/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)


### Train Set: features and target

In [ ]:
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

### Test Set: features and target

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

### Modelling pipeline

ML pipeline for predicting House price sale

In [ ]:
best_regressor_pipeline 

In [ ]:
joblib.dump(value=best_regressor_pipeline, filename=f"{file_path}/best_regressor_pipeline.pkl")

### Most important features plot

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')

---

Good job! Clear cell's outputs, push to the repo using git commands and move on to the next notebook